In [121]:
import MetaTrader5 as mt5 
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller 
from datetime import datetime
import requests
import pandas as pd
import numpy as np
import warnings
import json
warnings.filterwarnings("ignore")
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'
def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')

In [40]:
import math

In [41]:
math.floor(20.5)

20

In [184]:
# To calculate dealing range LOW, divide current price by po3 number corresponding to trading style and take first two digits and multiply it by 
# the po3 number. that's your new low
# 3 == intraday / session trader, 4 == daily trader
eurusd = get_rates('EURUSD.a', mt5.TIMEFRAME_M30, 50000)

def po3_range(name, po3):
    sym_range = {}
    symbol = get_rates(name + '.a', mt5.TIMEFRAME_H1, 500)
    current_price = int(symbol['close'].iloc[-1] * 10000)
    res = (current_price / 3**po3) 
    res = int(str(res)[:2])
    sym_range[name + ' low'] = (res * 3**po3) / 1000 # dealing low
    sym_range[name + ' high'] = ((res * 3**po3) + 3**po3) / 1000 # dealing high
    return sym_range

In [185]:
from zoneinfo import ZoneInfo
from datetime import datetime, timedelta

In [186]:
import pandas as pd
import itertools

# Assuming 'eurusd' is your DataFrame indexed by datetime in 30-minute intervals

# Generate all combinations of hourly intervals
hours = [f"{hour:02d}:00" for hour in range(24)]
intervals = list(itertools.product(hours, repeat=2))

In [199]:
from datetime import datetime, timedelta

def generate_hourly_intervals(start_hour, end_hour):
    intervals = []
    start = datetime.strptime(f"{start_hour}:00", "%H:%M")
    end = datetime.strptime(f"{end_hour}:00", "%H:%M")

    while start < end:
        current_end = start + timedelta(hours=1)
        while current_end <= end:
            intervals.append((start.strftime("%H:%M"), current_end.strftime("%H:%M")))
            current_end += timedelta(hours=1)
        start += timedelta(hours=1)
    
    return intervals

# US market hours in 24-hour format (9 AM to 4 PM)
intervals = generate_hourly_intervals(9, 16)

In [198]:
# Define the time intervals
intervals = [
    ('01:00', '07:30'),
    ('08:00', '13:00'),
    # ('10:00', '11:00'),
    # ('11:00', '13:00')
]

In [200]:
# Initialize a dictionary to store the returns for each interval
interval_returns = {interval: [] for interval in intervals}

# Group by each day
grouped = eurusd.groupby(eurusd.index.date)
for date, group in grouped:
    for start, end in intervals:
        # Select the rows within the interval for that day
        interval_data = group.between_time(start, end)
        if not interval_data.empty:
            start_price = interval_data.iloc[0]['close']  # First candle in the interval
            end_price = interval_data.iloc[-1]['close']  # Last candle in the interval

            # Calculate the return for the interval
            interval_return = (end_price - start_price) / start_price
            interval_returns[(start, end)].append(interval_return)
        else:
            interval_returns[(start, end)].append(None)  # No data in the interval for this day

# Display the interval returns
for interval, returns in interval_returns.items():
    print(f"Interval {interval}: Number of data points = {len(returns)}")

Interval ('09:00', '10:00'): Number of data points = 1043
Interval ('09:00', '11:00'): Number of data points = 1043
Interval ('09:00', '12:00'): Number of data points = 1043
Interval ('09:00', '13:00'): Number of data points = 1043
Interval ('09:00', '14:00'): Number of data points = 1043
Interval ('09:00', '15:00'): Number of data points = 1043
Interval ('09:00', '16:00'): Number of data points = 1043
Interval ('10:00', '11:00'): Number of data points = 1043
Interval ('10:00', '12:00'): Number of data points = 1043
Interval ('10:00', '13:00'): Number of data points = 1043
Interval ('10:00', '14:00'): Number of data points = 1043
Interval ('10:00', '15:00'): Number of data points = 1043
Interval ('10:00', '16:00'): Number of data points = 1043
Interval ('11:00', '12:00'): Number of data points = 1043
Interval ('11:00', '13:00'): Number of data points = 1043
Interval ('11:00', '14:00'): Number of data points = 1043
Interval ('11:00', '15:00'): Number of data points = 1043
Interval ('11:

In [219]:
correlation_matrix = pd.DataFrame(interval_returns).corr()

In [222]:
high_correlations = pd.DataFrame(columns=['Start Interval', 'End Interval', 'Correlation'])

# Iterate over the correlation matrix
for row_label, row in correlation_matrix.iterrows():
    for col_label, value in row.items():
        if value > 0.6 and row_label != col_label:  # Filter for correlations > 0.6 and avoid self-correlation
            new_row = {'Start Interval': row_label, 'End Interval': col_label, 'Correlation': value}
            high_correlations.loc[len(high_correlations)] = new_row

# Display the high correlations DataFrame
high_correlations

,Start Interval,End Interval,Correlation
0,"(09:00, 10:00)","(09:00, 11:00)",0.731899
1,"(09:00, 10:00)","(09:00, 12:00)",0.639691
2,"(09:00, 11:00)","(09:00, 10:00)",0.731899
3,"(09:00, 11:00)","(09:00, 12:00)",0.855395
4,"(09:00, 11:00)","(09:00, 13:00)",0.775326
...,...,...,...
211,"(14:00, 16:00)","(15:00, 16:00)",0.851813
212,"(15:00, 16:00)","(11:00, 16:00)",0.647983
213,"(15:00, 16:00)","(12:00, 16:00)",0.706390
214,"(15:00, 16:00)","(13:00, 16:00)",0.777598


In [176]:
eurusd = get_rates('EURUSD.a', mt5.TIMEFRAME_H4, 1000)

In [178]:
# Initialize a DataFrame to store returns for each interval
interval_returns = []

# Calculate returns for each interval
for i in range(1, len(eurusd)):
    start_price = eurusd.iloc[i - 1]['close']  # Previous interval's closing price
    end_price = eurusd.iloc[i]['close']        # Current interval's closing price

    # Calculate the return for the interval
    interval_return = (end_price - start_price) / start_price
    interval_returns.append(interval_return)
    

# Convert the list of returns to a DataFrame
interval_returns_df = pd.DataFrame(interval_returns, columns=['Return'])


In [179]:
interval_returns_df

,Return
0,-0.002781
1,0.001399
2,-0.001143
3,0.001171
4,0.001524
...,...
994,-0.000127
995,0.000709
996,0.002190
997,-0.001170
